In [1]:
import os, sys , subprocess
import essentia.standard , essentia.streaming

2023-03-21 22:27:39.563637: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
SERVER_TEST_PATH = '/raid/DATASETS/anomaly/XD_Violence/testing'
SERVER_TEST_AUD_PATH = '/raid/DATASETS/anomaly/XD_Violence/aud/testing'

In [3]:
def load_xdv_test():

    mp4_fn, mp4_labels = [],[]
    for root, dirs, files in os.walk(SERVER_TEST_PATH):
        for file in files:
            if file.find('.mp4') != -1:
                mp4_fn.append(os.path.join(root, file))
                if 'label_A' in  file:mp4_labels.append(0)
                else:mp4_labels.append(1)
                
    aac_fn, aac_labels = [],[]                            
    for root, dirs, files in os.walk(SERVER_TEST_AUD_PATH):
        for file in files:
            if file.find('.aac') != -1:
                aac_fn.append(os.path.join(root, file))
                if 'label_A' in  file:aac_labels.append(0)
                else:aac_labels.append(1)                
    
    return mp4_fn,mp4_labels,aac_fn,aac_labels

mp4_fn,mp4_labels,aac_fn,aac_labels = load_xdv_test()

In [4]:
def get_acodec_from_mp4(data):
    for i in range(len(data)):
        print(os.system('ffprobe -hide_banner -v error -select_streams a:0 -show_entries stream=codec_name,channels,sample_rate,bit_rate -of default=noprint_wrappers=1 -of compact=p=0:nk=1 '+str('"'+data[i]+'"')))
    #os.system('ffprobe -hide_banner -show_streams -select_streams a:0 -i '+str('"'+test_fn[1]+'"'))
    #os.system('ffprobe -hide_banner '+str('"'+test_fn[1]+'"'))

#get_acodec_from_mp4(mp4_fn)
#get_acodec_from_mp4(aac_fn)

In [5]:
def conv_mp42aac(data,dry_run=True):
    print(SERVER_TEST_AUD_PATH)
    for i in range(len(data)):
        out_fn = os.path.join(SERVER_TEST_AUD_PATH,os.path.splitext(os.path.basename(data[i]))[0]+'.aac')
        print("\nconverting ",os.path.basename(data[i]),'\n\t\t\t',os.path.basename(out_fn))

        if not dry_run:
            command = "ffmpeg -hide_banner -v warning -i "+str('"'+data[i]+'"')+" -c:a copy "+str('"'+out_fn+'"')
            os.system(command)

conv_mp42aac(mp4_fn,False)

In [6]:
print(dir(essentia.standard))
print(help(essentia.standard.MonoLoader))